In [1]:
from bs4 import BeautifulSoup


In [2]:
from playwright.async_api import async_playwright

In [3]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)

In [4]:
page = await browser.new_page()

In [5]:
await page.goto("https://eemis.mglsd.go.ug/companies")

<Response url='https://eemis.mglsd.go.ug/companies' request=<Request url='https://eemis.mglsd.go.ug/companies' method='GET'>>

In [6]:
html = await page.content()

In [7]:
soup_doc = BeautifulSoup(html)

In [8]:
#print(soup_doc.prettify())

In [9]:
while True:
    try:
        await page.wait_for_selector('a.load_more_comp', timeout=20000)
        await page.click('a.load_more_comp')
        print("Clicked 'Load More' button.")
        await page.wait_for_timeout(20000)
    except Exception as e:
        print("No more 'Load More' button or an error occurred:", e)
        break

Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
No more 'Load More' button or an error occurred: Page.wait_for_selector: Timeout 20000ms exceeded.
Call log:
waiting for locator("a.load_more_comp") to be visible



In [10]:
html = await page.content()
#html

In [11]:
soup_doc = BeautifulSoup(html)
#print(soup_doc.prettify())

In [12]:
companies_html = soup_doc.find_all('div',class_='col-md-6')
#companies_html

In [13]:
companies_list = []

for company in companies_html:
    company_dict = {}
    company_name = company.find('h4').text
    company_dict['company_name'] = company_name
    company_link = company.find('h4').find('a')['href']
    company_dict['company_link'] = company_link
    company_location = company.find('i',class_='fa fa-map-marker').next.text
    company_dict['company_location_companiespage'] = company_location
    company_phone = company.find('i',class_='fa fa-phone').next.text   
    company_dict['company_phone'] = company_phone
    company_calendar = company.find('i',class_='fa fa-calendar').next.text
    company_dict['company_calendar'] = company_calendar
    companies_list.append(company_dict)

companies_list[0:5]

[{'company_name': 'RIGHTFORCE MANPOWER SERVICES L.L.C',
  'company_link': 'https://eemis.mglsd.go.ug/companies/RIGHTFORCEMANPOWERSERVICESLLC_33389286',
  'company_location_companiespage': 'As Sulimaniyah, Riyadh 12431, Saudi Arabia',
  'company_phone': '+971-502-258556',
  'company_calendar': 'Jan 06, 2025 - Jan 06, 2027'},
 {'company_name': 'FIXIT MANPOWER SUPPLY L.L.C.',
  'company_link': 'https://eemis.mglsd.go.ug/companies/FixitManpowerSupplyLLC_33332694',
  'company_location_companiespage': 'As Sulimaniyah, Riyadh 12431, Saudi Arabia',
  'company_phone': '+971-524-326554',
  'company_calendar': 'Dec 12, 2024 - Dec 12, 2026'},
 {'company_name': 'B H R HUMAN RESOURCES CONSULTANCIES L.L.C',
  'company_link': 'https://eemis.mglsd.go.ug/companies/BHRHUMANRESOURCESCONSULTANCIESLLC_33247896',
  'company_location_companiespage': 'As Sulimaniyah, Riyadh 12431, Saudi Arabia',
  'company_phone': '+971-585-177862',
  'company_calendar': 'Dec 12, 2024 - Dec 12, 2026'},
 {'company_name': 'MEHAN

In [14]:
import pandas as pd

df = pd.json_normalize(companies_list)
#df.to_csv("companies_list.csv")

In [15]:
import requests

html = requests.get("https://eemis.mglsd.go.ug/companies/SAFIRALRYADAHRECRUITMENT_32994150").content

In [16]:
soup_doc = BeautifulSoup(html, "html.parser")
#print(soup_doc.prettify())

In [17]:
#getting info for one company page- testing out code

logo_link = soup_doc.find('div',class_='col-md-12 col-sm-12').find('img')['src']
company_name = soup_doc.find('div',class_='col-md-12 col-sm-12').find('div',class_='text-col').a.text

page_info = soup_doc.find('div',class_='col-md-12 col-sm-12').find_all('li')
for info in page_info:
    key = info.strong.string
    print(key)
    if info.br:
        value = info.strong.next_sibling, info.br.next
    else:
        value = info.strong.next_sibling
    print(value)

Licence No :
SA24110038
Start Date :
30 November 2024
Expiry Date :
29 November 2026
Address :
(' P.O Box ', 'Riyadh, Saudi Arabia ')
Phone :
 +966-533-894555 
Email :
 safiralryadh@gmail.com 


In [18]:
import requests
import time

for company in companies_list:
    time.sleep(2)
    try: 
        html = requests.get(company['company_link']).content
        soup_doc = BeautifulSoup(html, "html.parser")

        page_content = soup_doc.find('div',class_='col-md-12 col-sm-12')
        #print(page_content)

        logo_link = page_content.find('img')['src']
        #print(logo_link)
        company['logo_link'] = logo_link
        
        company_name_copg = page_content.find('div',class_='text-col').a.text
        company['company_name_copg'] = company_name_copg
        #print(company_name_copg)

        page_info = soup_doc.find('div',class_='col-md-12 col-sm-12').find_all('li')
        for info in page_info:
            key = info.strong.string
            #print(key)
            if info.br:
                value = info.strong.next_sibling, info.br.next
            else:
                value = info.strong.next_sibling
            #print(value)
            company[key] = value
    except:
        print("ERROR!")
    #print(company)

In [19]:
df = pd.json_normalize(companies_list)
df.head()

,company_name,company_link,company_location_companiespage,company_phone,company_calendar,logo_link,company_name_copg,Licence No :,Start Date :,Expiry Date :,Address :,Phone :,Email :
0,RIGHTFORCE MANPOWER SERVICES L.L.C,https://eemis.mglsd.go.ug/companies/RIGHTFORCE...,"As Sulimaniyah, Riyadh 12431, Saudi Arabia",+971-502-258556,"Jan 06, 2025 - Jan 06, 2027",https://eemis.mglsd.go.ug/assets/images/brands...,RIGHTFORCE MANPOWER SERVICES L.L.C,AE25010002,06 January 2025,06 January 2027,"( P.O Box , Dubayy, United Arab Emirates )",+971-502-258556,salim@rightforcegroup.com
1,FIXIT MANPOWER SUPPLY L.L.C.,https://eemis.mglsd.go.ug/companies/FixitManpo...,"As Sulimaniyah, Riyadh 12431, Saudi Arabia",+971-524-326554,"Dec 12, 2024 - Dec 12, 2026",https://eemis.mglsd.go.ug/assets/images/brands...,FIXIT MANPOWER SUPPLY L.L.C.,AE24120014,12 December 2024,12 December 2026,"( P.O Box , Dubayy, United Arab Emirates )",+971-524-326554,info@fixitmanpower.com
2,B H R HUMAN RESOURCES CONSULTANCIES L.L.C,https://eemis.mglsd.go.ug/companies/BHRHUMANRE...,"As Sulimaniyah, Riyadh 12431, Saudi Arabia",+971-585-177862,"Dec 12, 2024 - Dec 12, 2026",https://eemis.mglsd.go.ug/assets/images/brands...,B H R HUMAN RESOURCES CONSULTANCIES L.L.C,AE24120015,12 December 2024,12 December 2026,"( P.O Box , Dubayy, United Arab Emirates )",+971-585-177862,kritika@bharathrgroup.com
3,MEHAN HUMAN RESOURCES COMPANY,https://eemis.mglsd.go.ug/companies/MEHANHUMAN...,"Al khobar Dammam, Saudi Arabia",+996-539-115211,"Dec 11, 2024 - Dec 11, 2026",https://eemis.mglsd.go.ug/assets/images/brands...,MEHAN HUMAN RESOURCES COMPANY,SA24120010,11 December 2024,11 December 2026,"( P.O Box , Eastern Province, Saudi Arabia )",+996-539-115211,rec@mehan.sa
4,SHANKAR TRADING COMPANY LLC,https://eemis.mglsd.go.ug/companies/SHANKARTRA...,"As Sulimaniyah, Riyadh 12431, Saudi Arabia",+971-555-015120,"Dec 09, 2024 - Dec 09, 2026",https://eemis.mglsd.go.ug/assets/images/brands...,SHANKAR TRADING COMPANY LLC,AE24120009,09 December 2024,09 December 2026,"( P.O Box , Dubayy, United Arab Emirates )",+971-555-015120,adronabirungi3@gmail.com


In [20]:
df.to_csv("companies.csv", index=False)